In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ELT_PySpark") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.2,com.amazonaws:aws-java-sdk-bundle:1.11.901") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "your username") \
    .config("spark.hadoop.fs.s3a.secret.key", "your password") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

25/05/26 13:10:46 WARN Utils: Your hostname, lenovo-PC resolves to a loopback address: 127.0.1.1; using 172.31.112.46 instead (on interface eth0)
25/05/26 13:10:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntucoy/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntucoy/.ivy2/cache
The jars for the packages stored in: /home/ubuntucoy/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-da181f05-115c-4481-a617-5f730e43ebde;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 1499ms :: artifacts dl 44ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.2 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	:: evicted modules:
	com.amazonaws#aws-java-sdk-bundle;1.11.901 by [com.amazonaws#aws-java-sdk-bundle;1.11.1026] in [default]
	-----------------------------------------------------

In [2]:
df = spark.read.csv("s3a://raw/erp/LOC_A101.csv", header=True, inferSchema=True)
df.show()

25/05/26 13:11:22 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

+-----------+---------+
|        CID|    CNTRY|
+-----------+---------+
|AW-00011000|Australia|
|AW-00011001|Australia|
|AW-00011002|Australia|
|AW-00011003|Australia|
|AW-00011004|Australia|
|AW-00011005|Australia|
|AW-00011006|Australia|
|AW-00011007|Australia|
|AW-00011008|Australia|
|AW-00011009|Australia|
|AW-00011010|Australia|
|AW-00011011|Australia|
|AW-00011012|       US|
|AW-00011013|       US|
|AW-00011014|       US|
|AW-00011015|       US|
|AW-00011016|       US|
|AW-00011017|Australia|
|AW-00011018|Australia|
|AW-00011019|   Canada|
+-----------+---------+
only showing top 20 rows



In [3]:
from pyspark.sql.functions import col, trim, when, expr

# Transformation
df_transformed = df.select(
    expr("REPLACE(cid, '-', '')").alias("cid"),
    when(trim(col("cntry")) == "DE", "Germany")
    .when(trim(col("cntry")).isin("US", "USA"), "United States")
    .when((trim(col("cntry")) == "") | col("cntry").isNull(), "n/a")
    .otherwise(trim(col("cntry"))).alias("cntry")
)

df_transformed.show()

+----------+-------------+
|       cid|        cntry|
+----------+-------------+
|AW00011000|    Australia|
|AW00011001|    Australia|
|AW00011002|    Australia|
|AW00011003|    Australia|
|AW00011004|    Australia|
|AW00011005|    Australia|
|AW00011006|    Australia|
|AW00011007|    Australia|
|AW00011008|    Australia|
|AW00011009|    Australia|
|AW00011010|    Australia|
|AW00011011|    Australia|
|AW00011012|United States|
|AW00011013|United States|
|AW00011014|United States|
|AW00011015|United States|
|AW00011016|United States|
|AW00011017|    Australia|
|AW00011018|    Australia|
|AW00011019|       Canada|
+----------+-------------+
only showing top 20 rows



In [4]:
df_transformed.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("s3a://clean/erp/LOC_A101_clean.csv")

25/05/26 13:11:51 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
25/05/26 13:11:53 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
                                                                                

In [5]:
df_cleaned = spark.read.option("header", True).csv("s3a://clean/erp/LOC_A101_clean.csv")
df_cleaned.show()

+----------+-------------+
|       cid|        cntry|
+----------+-------------+
|AW00011000|    Australia|
|AW00011001|    Australia|
|AW00011002|    Australia|
|AW00011003|    Australia|
|AW00011004|    Australia|
|AW00011005|    Australia|
|AW00011006|    Australia|
|AW00011007|    Australia|
|AW00011008|    Australia|
|AW00011009|    Australia|
|AW00011010|    Australia|
|AW00011011|    Australia|
|AW00011012|United States|
|AW00011013|United States|
|AW00011014|United States|
|AW00011015|United States|
|AW00011016|United States|
|AW00011017|    Australia|
|AW00011018|    Australia|
|AW00011019|       Canada|
+----------+-------------+
only showing top 20 rows



In [6]:
import duckdb

# Convert from Spark to Pandas
df_cleaned_LOC_A101 = df_cleaned.toPandas()

# Connect to DuckDB
con = duckdb.connect("/mnt/d/data_engineering/duckdb/db/dev.duckdb")

# Save as DuckDB table (view)
con.register("erp_LOC_A101_clean_view", df_cleaned_LOC_A101)

# Save as DuckDB table (not view)
con.execute("CREATE OR REPLACE TABLE erp_LOC_A101_clean AS SELECT * FROM erp_LOC_A101_clean_view")

In [7]:
con.execute("SELECT * FROM erp_LOC_A101_clean LIMIT 10").df().style.hide(axis="index")

cid,cntry
AW00011000,Australia
AW00011001,Australia
AW00011002,Australia
AW00011003,Australia
AW00011004,Australia
AW00011005,Australia
AW00011006,Australia
AW00011007,Australia
AW00011008,Australia
AW00011009,Australia


In [8]:
con.close()